In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.backends.cudnn.benchmark = True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torchmetrics tifffile

In [ ]:
import os
os.environ["OPENCV_LOG_LEVEL"] = "OFF"

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import transforms, models
import torchmetrics
import numpy as np
import cv2
import tifffile
from PIL import Image
import time

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Hyperparameters
BATCH_SIZE = 16
LEARNING_RATE = 0.001 # Slightly higher LR is okay for Transfer Learning (Head only)
EPOCHS = 15
IMG_SIZE = 224
NUM_CLASSES = 2  # Binary: Dead vs Alive
TRAIN_SPLIT = 0.7
VAL_SPLIT = 0.15

# Data transformations
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(180),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

# ==========================================

PATH_LIVE = "/content/drive/MyDrive/Training_Images/291225 live Hela Cells"
PATH_DEAD = "/content/drive/MyDrive/Training_Images/291225 Dead Hela Cells"
PATH_AGGR_1 = "/content/drive/MyDrive/Training_Images/Clumps and debris R4 tiff"
PATH_AGGR_2 = "/content/drive/MyDrive/Training_Images/R4 DPSC p11 Jan 30 _R4"

class CellDataset(Dataset):
    def __init__(self, data_sources, transform=None):
        self.samples = []
        self.transform = transform
        self.valid_exts = ('.tif', '.tiff', '.png', '.jpg', '.jpeg', '.bmp')

        print(f"\nScanning for images...")
        for folder_path, m_label, v_label in data_sources:
            if not os.path.isdir(folder_path):
                print(f"ERROR: Folder not found: {folder_path}")
                continue

            count = 0
            for root, _, files in os.walk(folder_path):
                for file in files:
                    if file.lower().endswith(self.valid_exts):
                        full_path = os.path.join(root, file)
                        self.samples.append((full_path, m_label, v_label))
                        count += 1
            print(f"Found {count} images in: {os.path.basename(folder_path)}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, m_label, v_label = self.samples[idx]
        final_image = None

        # METHOD 1: OpenCV
        try:
            img_array = cv2.imread(path, cv2.IMREAD_UNCHANGED)
            if img_array is not None:
                image_rgb = None
                if len(img_array.shape) == 2:
                    image_rgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
                elif len(img_array.shape) == 3:
                    image_rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

                if image_rgb is not None:
                    final_image = Image.fromarray(image_rgb)
        except Exception:
            pass

        # METHOD 2: Tifffile
        if final_image is None:
            try:
                img_array = tifffile.imread(path)
                if len(img_array.shape) == 3:
                    img_array = img_array[0]
                final_image = Image.fromarray(img_array).convert('RGB')
            except Exception:
                pass

        # METHOD 3: Pillow
        if final_image is None:
            try:
                final_image = Image.open(path).convert('RGB')
            except Exception:
                print(f"Could not load: {os.path.basename(path)}")
                final_image = Image.new('RGB', (IMG_SIZE, IMG_SIZE))

        if self.transform:
            final_image = self.transform(final_image)

        target = v_label if v_label != -1 else 0
        return final_image, target

# Config: (Path, Morphology, Viability)
data_config = [
    (PATH_LIVE, 0, 1),
    (PATH_DEAD, 0, 0),
    (PATH_AGGR_1, 1, -1),
    (PATH_AGGR_2, 1, -1)
]

full_dataset = CellDataset(data_config, transform=None)
singlet_indices = [i for i, x in enumerate(full_dataset.samples) if x[2] != -1]
singlet_dataset = torch.utils.data.Subset(full_dataset, singlet_indices)
print(f"Training on {len(singlet_dataset)} Singlet (Live/Dead) images.")

# Splits
train_size = int(TRAIN_SPLIT * len(singlet_dataset))
val_size = int(VAL_SPLIT * len(singlet_dataset))
test_size = len(singlet_dataset) - train_size - val_size
train_sub, val_sub, test_sub = random_split(singlet_dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42))

class TransformSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
    def __getitem__(self, idx):
        img, label = self.subset[idx]
        if self.transform:
            img = self.transform(img)
        return img, label
    def __len__(self):
        return len(self.subset)

train_data = TransformSubset(train_sub, transform=train_transforms)
val_data = TransformSubset(val_sub, transform=val_transforms)
test_data = TransformSubset(test_sub, transform=val_transforms)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# ==========================================
# 2. MODEL FACTORY (WITH FREEZING)
# ==========================================

# --- Custom SAM-Inspired Model (Trained from Scratch) ---
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        return self.sigmoid(avg_out + max_out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        padding = 3 if kernel_size == 7 else 1
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        return self.sigmoid(self.conv1(torch.cat([avg_out, max_out], dim=1)))

class SAMInspiredCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SAMInspiredCNN, self).__init__()
        self.backbone = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.ca = ChannelAttention(128)
        self.sa = SpatialAttention()
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, num_classes)

    def forward(self, x):
        # We don't freeze SAM model as it's "Custom"
        return self.backbone(x)

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def create_model(model_name, num_classes=2):
    print(f"Creating model: {model_name}...")
    model = None
    freeze = True # Default to Transfer Learning

    if model_name == 'sam_cnn':
        model = SAMInspiredCNN(num_classes=num_classes)
        freeze = False # Custom model should be trained fully

    elif model_name == 'vgg16':
        model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        set_parameter_requires_grad(model, freeze)
        model.classifier[6] = nn.Linear(4096, num_classes)

    elif model_name == 'vgg19':
        model = models.vgg19(weights=models.VGG19_Weights.DEFAULT)
        set_parameter_requires_grad(model, freeze)
        model.classifier[6] = nn.Linear(4096, num_classes)

    elif model_name == 'resnet18':
        model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        set_parameter_requires_grad(model, freeze)
        model.fc = nn.Linear(model.fc.in_features, num_classes)

    elif model_name == 'resnet34':
        model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
        set_parameter_requires_grad(model, freeze)
        in_ftrs = model.fc.in_features if isinstance(model.fc, nn.Linear) else 512
        model.fc = nn.Linear(in_ftrs, num_classes)

    elif model_name == 'resnet50':
        model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        set_parameter_requires_grad(model, freeze)
        in_ftrs = model.fc.in_features if isinstance(model.fc, nn.Linear) else 2048
        model.fc = nn.Linear(in_ftrs, num_classes)

    elif model_name == 'efficientnet_b0':
        model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        set_parameter_requires_grad(model, freeze)
        in_ftrs = model.classifier[1].in_features if isinstance(model.classifier[1], nn.Linear) else 1280
        model.classifier[1] = nn.Linear(in_ftrs, num_classes)

    elif model_name == 'efficientnet_b1':
        model = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.DEFAULT)
        set_parameter_requires_grad(model, freeze)
        in_ftrs = model.classifier[1].in_features if isinstance(model.classifier[1], nn.Linear) else 1280
        model.classifier[1] = nn.Linear(in_ftrs, num_classes)

    elif model_name == 'mobilenet_v2':
        model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
        set_parameter_requires_grad(model, freeze)
        in_ftrs = model.classifier[1].in_features if isinstance(model.classifier[1], nn.Linear) else 1280
        model.classifier[1] = nn.Linear(in_ftrs, num_classes)

    elif model_name == 'vit_b_16':
        model = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)
        set_parameter_requires_grad(model, freeze)
        in_ftrs = model.heads.head.in_features if isinstance(model.heads.head, nn.Linear) else 768
        model.heads.head = nn.Linear(in_ftrs, num_classes)

    elif model_name == 'swin_t':
        model = models.swin_t(weights=models.Swin_T_Weights.DEFAULT)
        set_parameter_requires_grad(model, freeze)
        in_ftrs = model.head.in_features if isinstance(model.head, nn.Linear) else 768
        model.head = nn.Linear(in_ftrs, num_classes)

    else:
        raise ValueError(f"Unknown model name: '{model_name}'. Choose from: vgg16, vgg19, resnet18, resnet34, resnet50, efficientnet_b0, efficientnet_b1, mobilenet_v2, vit_b_16, swin_t, sam_cnn")

    return model

def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

# ==========================================
# 3. TRAINING & EVALUATION LOOP
# ==========================================

def train_and_evaluate(model_name, epochs=EPOCHS):
    model = create_model(model_name, NUM_CLASSES).to(device)
    total_params, trainable_params = count_parameters(model)

    criterion = nn.CrossEntropyLoss()
    # Optimizer only updates parameters that require grad
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE, weight_decay=1e-4)

    # Initialize Metrics
    metric_acc = torchmetrics.Accuracy(task="binary").to(device)
    metric_prec = torchmetrics.Precision(task="binary").to(device)
    metric_rec = torchmetrics.Recall(task="binary").to(device)
    metric_f1 = torchmetrics.F1Score(task="binary").to(device)
    metric_auroc = torchmetrics.AUROC(task="binary").to(device)
    metric_confmat = torchmetrics.ConfusionMatrix(task="binary", num_classes=2).to(device)

    best_f1 = 0.0
    best_metrics = {}

    print(f"\nTraining {model_name} | Total Params: {total_params:,} | Trainable: {trainable_params:,}")

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        total_samples = 0 # Manual counter to fix len() error

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Manual stats update
            bs = inputs.size(0)
            running_loss += loss.item() * bs
            total_samples += bs

        train_loss = running_loss / total_samples if total_samples > 0 else 0

        # Validation
        model.eval()
        val_preds = []
        val_targets = []
        val_probs = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)

                probs = torch.softmax(outputs, dim=1)[:, 1]
                preds = torch.argmax(outputs, dim=1)

                val_preds.append(preds)
                val_targets.append(labels)
                val_probs.append(probs)

        if len(val_preds) > 0:
            val_preds = torch.cat(val_preds)
            val_targets = torch.cat(val_targets)
            val_probs = torch.cat(val_probs)

            acc = metric_acc(val_preds, val_targets).item() * 100
            prec = metric_prec(val_preds, val_targets).item() * 100
            rec = metric_rec(val_preds, val_targets).item() * 100
            f1 = metric_f1(val_preds, val_targets).item() * 100
            try:
                auroc = metric_auroc(val_probs, val_targets).item() * 100
            except:
                auroc = 0.0
        else:
            acc, prec, rec, f1, auroc = 0,0,0,0,0

        print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss:.4f} | Val F1: {f1:.2f}% | Val Acc: {acc:.2f}%")

        if f1 > best_f1:
            best_f1 = f1
            conf_matrix = metric_confmat(val_preds, val_targets).cpu().numpy()
            best_metrics = {
                "Accuracy": acc,
                "Precision": prec,
                "Recall": rec,
                "F1-Score": f1,
                "AUROC": auroc,
                "Confusion Matrix": conf_matrix,
                "Params": total_params,
                "Trainable": trainable_params
            }
            torch.save(model.state_dict(), f'/content/drive/MyDrive/Training_Images/best_{model_name}.pth')

    return best_metrics

# ==========================================
# 4. RUN BENCHMARK
# ==========================================

if __name__ == "__main__":
    models_to_run = [
        'resnet18'
    ]

    results = {}

    for m in models_to_run:
        try:
            metrics = train_and_evaluate(m, epochs=EPOCHS)
            results[m] = metrics
        except Exception as e:
            print(f"Failed to run {m}: {e}")
            import traceback
            traceback.print_exc()

    print("\n" + "="*90)
    print(f"{'MODEL':<20} | {'PARAMS (M)':<10} | {'ACC':<7} | {'PREC':<7} | {'REC':<7} | {'F1':<7} | {'AUROC':<7}")
    print("="*90)

    for m, metrics in results.items():
        if metrics:
            params_m = metrics['Params'] / 1e6
            print(f"{m:<20} | {params_m:<10.1f} | {metrics['Accuracy']:<7.2f} | {metrics['Precision']:<7.2f} | {metrics['Recall']:<7.2f} | {metrics['F1-Score']:<7.2f} | {metrics['AUROC']:<7.2f}")

    print("\nDetailed Confusion Matrices:")
    for m, metrics in results.items():
        if metrics:
            print(f"\n--- {m} ---")
            print(metrics['Confusion Matrix'])